# SFT--Supervised Fine-Turning

1. 背景：预训练模型（如 GPT、LLaMA）  
大模型通常先通过无监督预训练（如语言建模）在海量文本上学习通用语言表示，具备广泛的语言理解和生成能力。  
2. SFT 的目标  
通过有标签的数据（输入-输出对）对预训练模型进行微调，使其在特定任务或行为上表现更好，比如：  
* 对话系统（如 ChatGPT）
* 问答系统
* 指令跟随（instruction following）

1.数据处理

In [1]:
!pip install --upgrade datasets huggingface_hub

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple


In [2]:
import os
os.environ["HF_ENDPOINT"] = "https://hf-mirror.com"

In [3]:
from datasets import load_dataset

test_dataset = load_dataset("json", data_files="C:/Users/hhm18/Desktop/深度学习/env_LLM/firefly-train-1.1M.jsonl", split="train[:500]")

C:\Users\hhm18\miniconda3\envs\env_LLM\lib\site-packages\requests\__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(


In [4]:
test_dataset

Dataset({
    features: ['kind', 'input', 'target'],
    num_rows: 500
})

In [5]:
test_dataset[400]

{'kind': 'JinYongGeneration',
 'input': '武侠小说续写：\n吴坎？”万圭道：“是！我在后园中见这贱人藏好剑谱，便远远地跟着',
 'target': '她，哪知道她……她到了回廊上，竟和吴坎那厮勾勾搭搭，这淫妇……好不要脸！”万震山沉吟道：“我看她平素为人倒也规矩端正，不像是这样子的人。你没瞧错么？他二人说些什么？”万圭道：“孩儿怕他们知觉，不敢走得太近，回廊上没隐蔽的地方，只有躲在墙角后面。这两个狗男女说话很轻，没能完全听到，可是……可是也听到了大半。”\n\u3000\u3000万震山“嗯”了一声，道：“孩儿，你别气急。大丈夫何患无妻？咱们既得了剑谱，又奄明了这中间的秘密，转眼便可富甲天下，你便要买一百个姬妾，那也容易得紧。你坐下，慢慢地说！”\n\u3000\u3000只听得床板格格两响，万圭坐到了床上，气喘喘地道：“那淫妇藏好书本，很是得意，嘴里居然哼着小曲。那奸夫一见到她，满脸堆欢，说道：‘今晚三更，我在柴房中等你，可别忘了！’的的确确是这几句话，我听得清清楚楚的。”万震山怒道：“那小淫妇又怎么说？”万圭道：“她……她说道：‘没好死的，狗胆子这么大，连命也不要了！’”\n\u3000\u3000戚芳在窗外只听得心乱如麻：“他……他二人口口声声地骂我淫妇，怎……怎么能如此地冤枉人家？三哥，我是一片为你之心，要夺回解药，治你之伤。你却这般辱我，可还有良心没有？”\n\u3000\u3000只听万圭续道：“我……我听了他们这么说，心头火起，恨不得拔剑上前将二人杀了。只是我没带剑，又伤后没力，不能跟他们明争，当即赶回房去，免得那贼淫妇回房时不见到我，起了疑心。奸夫淫妇以后再说什么，我就没再听见。”万震山道：“哼，有其父必有其女，果然一门都是无耻之辈。咱们先去取了剑谱，再到柴房外守候。捉奸捉双，叫这对狗男女死而无怨！”\n\u3000\u3000万圭道：“那淫妇恋奸情热，等不到三更天，早就出去了，这会儿……这会儿……”说着牙齿咬得格格直响。万震山道：“那么咱们即刻便去。你拿好了剑，可先别出手，等我斩断他二人的手足，再由你亲手取这双狗男女的性命。”\n\u3000\u3000只见房门推开，万震山左手托在万圭腋下，二人径奔后园。\n\u3000\u3000戚芳靠在墙上，眼泪扑簌簌地从衣襟上滚下来。她只盼治好丈夫的伤，他却对自己如此

In [6]:
from transformers import AutoModelForCausalLM, AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen2.5-0.5B-Instruct", trust_remote_code=True)


In [7]:
print(tokenizer.chat_template)

{%- if tools %}
    {{- '<|im_start|>system\n' }}
    {%- if messages[0]['role'] == 'system' %}
        {{- messages[0]['content'] }}
    {%- else %}
        {{- 'You are Qwen, created by Alibaba Cloud. You are a helpful assistant.' }}
    {%- endif %}
    {{- "\n\n# Tools\n\nYou may call one or more functions to assist with the user query.\n\nYou are provided with function signatures within <tools></tools> XML tags:\n<tools>" }}
    {%- for tool in tools %}
        {{- "\n" }}
        {{- tool | tojson }}
    {%- endfor %}
    {{- "\n</tools>\n\nFor each function call, return a json object with function name and arguments within <tool_call></tool_call> XML tags:\n<tool_call>\n{\"name\": <function-name>, \"arguments\": <args-json-object>}\n</tool_call><|im_end|>\n" }}
{%- else %}
    {%- if messages[0]['role'] == 'system' %}
        {{- '<|im_start|>system\n' + messages[0]['content'] + '<|im_end|>\n' }}
    {%- else %}
        {{- '<|im_start|>system\nYou are Qwen, created by Alibaba C

需要处理成输入格式一致  

<|im_start|>system  
我是一个非常棒的人工智能助手。<|im_end|>  
<|im_start|>user  
可以简单介绍你自己吗？<|im_end|>  
<|im_start|>assistant  
我是Xxxxxxxxxxxxxxxxxxx。<|im_end|>  

In [8]:
def format_prompt(example):
    chat = [
        {"role": "system", "content": "我是一个非常棒的人工智能助手。"},
        {"role": "user", "content": example["input"]},
        {"role": "assistant", "content": example["target"]}
    ]
    prompt = tokenizer.apply_chat_template(chat, tokenize=False)
    return {"text": prompt}

dataset = test_dataset.map(format_prompt, remove_columns=test_dataset.column_names)
dataset

Dataset({
    features: ['text'],
    num_rows: 500
})

In [9]:
dataset[0]

{'text': '<|im_start|>system\n我是一个非常棒的人工智能助手。<|im_end|>\n<|im_start|>user\n自然语言推理：\n前提：家里人心甘情愿地养他,还有几家想让他做女婿的\n假设：他是被家里人收养的孤儿<|im_end|>\n<|im_start|>assistant\n中立<|im_end|>\n'}

2.加载模型

In [10]:
model = AutoModelForCausalLM.from_pretrained("Qwen/Qwen2.5-0.5B-Instruct", 
                                             trust_remote_code=True)
tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen2.5-0.5B-Instruct", 
                                          trust_remote_code=True)
tokenizer.padding_side = "left" # 自回归模型（只能从左到右看），设置padding在左侧

3.配置参数

In [11]:
print(model)

Qwen2ForCausalLM(
  (model): Qwen2Model(
    (embed_tokens): Embedding(151936, 896)
    (layers): ModuleList(
      (0-23): 24 x Qwen2DecoderLayer(
        (self_attn): Qwen2Attention(
          (q_proj): Linear(in_features=896, out_features=896, bias=True)
          (k_proj): Linear(in_features=896, out_features=128, bias=True)
          (v_proj): Linear(in_features=896, out_features=128, bias=True)
          (o_proj): Linear(in_features=896, out_features=896, bias=False)
        )
        (mlp): Qwen2MLP(
          (gate_proj): Linear(in_features=896, out_features=4864, bias=False)
          (up_proj): Linear(in_features=896, out_features=4864, bias=False)
          (down_proj): Linear(in_features=4864, out_features=896, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): Qwen2RMSNorm((896,), eps=1e-06)
        (post_attention_layernorm): Qwen2RMSNorm((896,), eps=1e-06)
      )
    )
    (norm): Qwen2RMSNorm((896,), eps=1e-06)
    (rotary_emb): Qwen2RotaryEmbe

In [12]:
from peft import LoraConfig, get_peft_model

peft_config = LoraConfig(
    lora_alpha=32,
    lora_dropout=0.1,
    r=64,
    target_modules=["q_proj", "k_proj", "v_proj"],
    task_type="CAUSAL_LM",
    bias="none",
)

model = get_peft_model(model, peft_config)

4.训练配置项

In [13]:
from transformers import TrainingArguments

output_dir = "./results"

# Traning arg
training_args = TrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=1,  
    gradient_accumulation_steps=8,    #  实际的batch_size = 2 * 4 = 8
    num_train_epochs=1,
    learning_rate=2e-4,
    warmup_ratio=0.03,
    logging_steps=10,
    # max_seq_length=512,            # 训练时的最大序列长度
    save_strategy="steps",
    save_steps=20,                    # 每20步保存一次模型正常不会设置这么小
    save_total_limit=3,
    optim="adamw_torch",
    fp16=True,
    bf16=False,                       # 如果使用A100等支持BF16的GPU，可以设置为True
)

In [14]:
!pip install -U trl

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple


In [15]:
model.gradient_checkpointing_enable()

In [16]:
from trl import SFTTrainer

trainer = SFTTrainer(
    model=model,
    args=training_args,
    train_dataset=dataset,
    processing_class=tokenizer,
    peft_config=peft_config,
)

trainer.train()

trainer.model.save_pretrained("C:/Users/hhm18/Desktop/深度学习/env_LLM/result/final_result")

C:\Users\hhm18\miniconda3\envs\env_LLM\lib\site-packages\peft\mapping_func.py:73: UserWarning: You are trying to modify a model with PEFT for a second time. If you want to reload the model with a different config, make sure to call `.unload()` before.
  warnings.warn(
C:\Users\hhm18\miniconda3\envs\env_LLM\lib\site-packages\peft\mapping_func.py:79: UserWarning: The PEFT config's `base_model_name_or_path` was renamed from 'Qwen/Qwen2.5-0.5B-Instruct' to 'None'. Please ensure that the correct base model is loaded when loading this checkpoint.
  warnings.warn(
C:\Users\hhm18\miniconda3\envs\env_LLM\lib\site-packages\peft\tuners\tuners_utils.py:196: UserWarning: Already found a `peft_config` attribute in the model. This will lead to having multiple adapters in the model. Make sure to know what you are doing!
  warnings.warn(
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.
C:\Users\hhm18\miniconda3\envs\env_LLM\lib\site-packages\transformers\integrat

Step,Training Loss
10,3.829200
20,3.462200
30,3.135700
40,2.916400
50,3.002600
60,2.941800


4.*Merge Adepter 

In [17]:
# 查看可训练的参数
trainer.model.print_trainable_parameters()

trainable params: 5,898,240 || all params: 499,931,008 || trainable%: 1.1798


In [19]:
from transformers import AutoModelForCausalLM
from peft import PeftModel

base_model = AutoModelForCausalLM.from_pretrained("Qwen/Qwen2.5-0.5B-Instruct", trust_remote_code=True)
model = PeftModel.from_pretrained(base_model, "C:/Users/hhm18/Desktop/深度学习/env_LLM/result/final_result")
merged_model = model.merge_and_unload()
merged_model.save_pretrained("C:/Users/hhm18/Desktop/深度学习/env_LLM/result/merged_model")

C:\Users\hhm18\miniconda3\envs\env_LLM\lib\site-packages\peft\peft_model.py:585: UserWarning: Found missing adapter keys while loading the checkpoint: ['base_model.model.model.layers.0.self_attn.q_proj.lora_A.default.weight', 'base_model.model.model.layers.0.self_attn.q_proj.lora_B.default.weight', 'base_model.model.model.layers.0.self_attn.k_proj.lora_A.default.weight', 'base_model.model.model.layers.0.self_attn.k_proj.lora_B.default.weight', 'base_model.model.model.layers.0.self_attn.v_proj.lora_A.default.weight', 'base_model.model.model.layers.0.self_attn.v_proj.lora_B.default.weight', 'base_model.model.model.layers.1.self_attn.q_proj.lora_A.default.weight', 'base_model.model.model.layers.1.self_attn.q_proj.lora_B.default.weight', 'base_model.model.model.layers.1.self_attn.k_proj.lora_A.default.weight', 'base_model.model.model.layers.1.self_attn.k_proj.lora_B.default.weight', 'base_model.model.model.layers.1.self_attn.v_proj.lora_A.default.weight', 'base_model.model.model.layers.1.s

In [20]:
merged_model = model.merge_and_unload()

In [23]:
from transformers import pipeline
pipe = pipeline("text-generation", model=merged_model, tokenizer=tokenizer, trust_remote_code=True)

promopt = """<|im_start|>system  
我是一个非常棒的人工智能助手。<|im_end|>  
<|im_start|>user  
使用翻译成文言文：等到对得到或喜爱的东西已经厌倦，感情随着事物的变化而变化，感慨随之产生。<|im_end|>  
<|im_start|>assistant  
"""

print(pipe(promopt, 
           max_new_tokens=128, 
           do_sample=True, 
           temperature=0.7, 
           top_p=0.9, 
           top_k=50)[0]["generated_text"])

Device set to use cuda:0


<|im_start|>system  
我是一个非常棒的人工智能助手。<|im_end|>  
<|im_start|>user  
使用翻译成文言文：等到对得到或喜爱的东西已经厌倦，感情随着事物的变化而变化，感慨随之产生。<|im_end|>  
<|im_start|>assistant  
等到得物已厌倦，情志随事转移；慨叹随之兴起。


In [24]:
promopt = """<|im_start|>system  
我是一个非常棒的人工智能助手。<|im_end|>  
<|im_start|>user  
你刚才经过SFT学习到了什么新东西啊？<|im_end|>  
<|im_start|>assistant  
"""

print(pipe(promopt, 
           max_new_tokens=128, 
           do_sample=True, 
           temperature=0.7, 
           top_p=0.9, 
           top_k=50)[0]["generated_text"])

<|im_start|>system  
我是一个非常棒的人工智能助手。<|im_end|>  
<|im_start|>user  
你刚才经过SFT学习到了什么新东西啊？<|im_end|>  
<|im_start|>assistant  
你好！作为一个人工智能，我没有能力学习或记忆知识。我的“学习”只是通过接收和处理数据来提供信息和服务。如果你有任何问题需要解答，请告诉我。
